<a href="https://colab.research.google.com/github/SteinwayThor/eeg-features-movies/blob/main/EDA_and_Data_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# generic torch imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchsummary import summary

# generic ml/stats imports
import cv2
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import scipy.fftpack as sp
from sklearn.model_selection import train_test_split
from math import pi
from scipy.spatial.distance import pdist, squareform
import pandas as pd

#torch vision related imports
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision import datasets

# data utils
import time
import os
import pickle
import json
from PIL import Image
from google.colab import drive
import bz2
import _pickle as cPickle
import pickle as pl
from scipy.io import loadmat

In [3]:
drive.mount('/content/drive', force_remount=True)

# Load any compressed pickle file
def decompress_pickle(file):
  data = bz2.BZ2File(file, 'rb')
  data = cPickle.load(data)
  return data

Mounted at /content/drive


In [4]:
emotion_annotations_f = '/content/drive/MyDrive/Movie Data/e0018RI/emotion_annotations'
emotion_annotations = os.listdir(emotion_annotations_f)
#skyfall_emotion_[angry|fear|sad|happy|disgust]_viewer[123|127|139|142]_chapter[01 to 31].json

In [5]:
#EDA on emotions json data
diff_emotions = {}
diff_viewers = {}
diff_chapters = {}
for json_file in emotion_annotations:
  split_name = json_file.split("_")
  emotion = split_name[2]  #angry
  viewer = split_name[3]  #viewer127
  chapter = split_name[4][:split_name[4].find(".")]  #chapter01
  diff_emotions[emotion] = diff_emotions.get(emotion,[]) + [json_file]
  diff_viewers[viewer] = diff_viewers.get(viewer,[]) + [json_file]
  diff_chapters[chapter] = diff_chapters.get(chapter,[]) + [json_file]
print(sorted(diff_chapters.keys()), diff_emotions.keys(), diff_viewers.keys(), sep='\n-------\n')

['chapter01', 'chapter02', 'chapter03', 'chapter04', 'chapter05', 'chapter06', 'chapter07', 'chapter08', 'chapter09', 'chapter10', 'chapter11', 'chapter12', 'chapter13', 'chapter14', 'chapter15', 'chapter16', 'chapter17', 'chapter18', 'chapter19', 'chapter20', 'chapter21', 'chapter22', 'chapter23', 'chapter24', 'chapter25', 'chapter26', 'chapter27', 'chapter28', 'chapter29', 'chapter30', 'chapter31']
-------
dict_keys(['disgust', 'angry', 'fear', 'sad', 'happy'])
-------
dict_keys(['viewer142', 'viewer127', 'viewer123', 'viewer139'])


In [6]:
#global insight:
# the file name is in the format: skyfall_emotion_[angry|fear|sad|happy|disgust]_viewer[123|127|139|142]_chapter[01 to 31].json

print("\n======================================================================")
print("Total different CHAPTERS for which labels are available along with their count: ")
print("Total CHAPTERS: ", len(diff_chapters.keys()))
chapter_plot = []
for chapter in sorted(list(diff_chapters.keys())):
  print(chapter, ", number of label files: ", len(diff_chapters[chapter]))
  print("Files:", sorted(diff_chapters[chapter]))
  chapter_plot.append([chapter,len(diff_chapters[chapter])])
chapter_plot = np.array(chapter_plot)
#insights:
# 1. json files for a single chapter represent different emotions corresponding to each file.
# 2. There can be min of 2 and max of 4 emotions/json per chapter.
# 3. Chapter 32 has no emotions data for it (end credits). Implication: Ignore neural data for it.
# 4. We can ignore viewer number in the file name here.

emotion_plot = []
print("\n======================================================================\n")
print("Total different EMOTIONS for which labels are available along with their count: ")
print("Total EMOTIONS: ", len(diff_emotions.keys()))
for emotion in sorted(list(diff_emotions.keys())):
  print(emotion, ", number of label files: ", len(diff_emotions[emotion]), end=',')
  print("Files:", sorted(diff_emotions[emotion]))
  emotion_plot.append([emotion, len(diff_emotions[emotion])])
emotion_plot = np.array(emotion_plot)
#insights:
# 1. 5 total emotions: angry, fear, happy, disgust and sad.
# 2. There is class-imbalance in the emotion classes.
# 3. Most of the data corresponds to negative emotions, possibly because skyfall has a lot of sad and intense scenes.

viewer_plot = []
print("\n======================================================================\n")
print("Total different VIEWERS for which labels are available along with their count: ")
print("Total VIEWERS: ", len(diff_viewers.keys()))
for viewer in sorted(list(diff_viewers.keys())):
  print(viewer, ", number of label files: ", len(diff_viewers[viewer]))
  print("Files:", sorted(diff_viewers[viewer]))
  viewer_plot.append([viewer, len(diff_viewers[viewer])])
viewer_plot = np.array(viewer_plot)
#insights:
# 1. There are 4 distinct viewers but for our purposes, we can just ignore viewer differences in annotation. Assuming no bias in viewers' annotations.


Total different CHAPTERS for which labels are available along with their count: 
Total CHAPTERS:  31
chapter01 , number of label files:  4
Files: ['skyfall_emotion_angry_viewer127_chapter01.json', 'skyfall_emotion_fear_viewer127_chapter01.json', 'skyfall_emotion_happy_viewer139_chapter01.json', 'skyfall_emotion_sad_viewer123_chapter01.json']
chapter02 , number of label files:  4
Files: ['skyfall_emotion_angry_viewer127_chapter02.json', 'skyfall_emotion_fear_viewer127_chapter02.json', 'skyfall_emotion_happy_viewer139_chapter02.json', 'skyfall_emotion_sad_viewer123_chapter02.json']
chapter03 , number of label files:  4
Files: ['skyfall_emotion_angry_viewer127_chapter03.json', 'skyfall_emotion_fear_viewer127_chapter03.json', 'skyfall_emotion_happy_viewer139_chapter03.json', 'skyfall_emotion_sad_viewer123_chapter03.json']
chapter04 , number of label files:  4
Files: ['skyfall_emotion_angry_viewer127_chapter04.json', 'skyfall_emotion_fear_viewer127_chapter04.json', 'skyfall_emotion_happy_v

In [7]:
fig, axs = plt.subplots(3,1, figsize=(10,6))
fig.suptitle("Plots for frequency of data for chapters, emotions and viewers")
fig.subplots_adjust(hspace=0.6)

axs[0].bar(range(1,1+chapter_plot.shape[0]),chapter_plot[:,1].astype('int'), color = 'lawngreen')
axs[0].set_xticks(range(1,1+chapter_plot.shape[0]))
axs[0].set_yticks([1,2,3,4])
axs[0].set_ylabel("Emotions")
axs[0].set_xlabel("Chapters 1 to 31")

_ = axs[1].bar(range(1,1+emotion_plot.shape[0]),emotion_plot[:,1].astype('int'), color = 'lawngreen')
axs[1].bar_label(_, label_type='center')
axs[1].set_xticks(range(1,1+emotion_plot.shape[0]), emotion_plot[:,0])
axs[1].set_ylabel("Chapters")
axs[1].set_xlabel("Emotions")

_ = axs[2].bar(range(1,1+viewer_plot.shape[0]),viewer_plot[:,1].astype('int'), color = 'lawngreen')
axs[2].bar_label(_, label_type='center')
axs[2].set_xticks(range(1,1+viewer_plot.shape[0]), viewer_plot[:,0])
axs[2].set_ylabel("JSON Files")
axs[2].set_xlabel("Viewers")
plt.plot()

[]

Extract and sync emotion labels with frame IDs

In [8]:
#chapter wise data extractions
#all the label data for a chapter.
all_data = {}
#key: chapter name
#value: list of json files loaded in/parsed
for x in diff_chapters:
  all_data[x] = []
  for label_file in diff_chapters[x]:
    all_data[x].append(json.load(open(os.path.join(emotion_annotations_f, label_file))))

In [9]:
ch = np.array(['chapter' + str(i).zfill(2) for i in range(1,32)])
print(ch)

['chapter01' 'chapter02' 'chapter03' 'chapter04' 'chapter05' 'chapter06'
 'chapter07' 'chapter08' 'chapter09' 'chapter10' 'chapter11' 'chapter12'
 'chapter13' 'chapter14' 'chapter15' 'chapter16' 'chapter17' 'chapter18'
 'chapter19' 'chapter20' 'chapter21' 'chapter22' 'chapter23' 'chapter24'
 'chapter25' 'chapter26' 'chapter27' 'chapter28' 'chapter29' 'chapter30'
 'chapter31']


In [10]:
all_data.keys(), all_data[ch[0]][0].keys()

(dict_keys(['chapter20', 'chapter21', 'chapter22', 'chapter25', 'chapter23', 'chapter24', 'chapter26', 'chapter28', 'chapter29', 'chapter30', 'chapter27', 'chapter03', 'chapter01', 'chapter31', 'chapter02', 'chapter06', 'chapter04', 'chapter07', 'chapter05', 'chapter12', 'chapter11', 'chapter08', 'chapter10', 'chapter09', 'chapter14', 'chapter15', 'chapter16', 'chapter17', 'chapter13', 'chapter18', 'chapter19']),
 dict_keys(['tags', 'shapes', 'version', 'tracks']))

In [11]:
#All possible values of tags, version and shapes across json files for all chapters
#version: Use of version is not apparent, will figure out its role if the need be in the future, seems random for now.
#tags: Tags is just empty, ignore.
#shapes: only present in 4 chapters. is it useful?? don't think so.
#tracks: useful but what?
tags = [] #unique values of tags across data
versions = [] #unique values of version across data
shapes = {}  #checking for shapes key existence over the json files for each chapter
dict_keys = []
for chapter in all_data:
  tags.extend([len(all_data[chapter][i]['tags']) for i in range(len(all_data[chapter]))])
  versions.extend([all_data[chapter][i]['version'] for i in range(len(all_data[chapter]))])
  dict_keys.extend([len(all_data[chapter][i].keys()) for i in range(len(all_data[chapter]))])
  shapes[chapter] = max([len(all_data[chapter][i]['shapes']) for i in range(len(all_data[chapter]))])
print('versions:',set(versions), '#dict keys:',set(dict_keys), '#tags:',set(tags), 'shape present?',sorted(shapes.items(), key=lambda x: x[1], reverse=True), sep='\n')


versions:
{0, 2, 3, 5, 6, 8, 9, 11, 12, 14, 15, 17, 18, 20, 21}
#dict keys:
{4}
#tags:
{0}
shape present?
[('chapter23', 1), ('chapter26', 1), ('chapter28', 1), ('chapter01', 1), ('chapter20', 0), ('chapter21', 0), ('chapter22', 0), ('chapter25', 0), ('chapter24', 0), ('chapter29', 0), ('chapter30', 0), ('chapter27', 0), ('chapter03', 0), ('chapter31', 0), ('chapter02', 0), ('chapter06', 0), ('chapter04', 0), ('chapter07', 0), ('chapter05', 0), ('chapter12', 0), ('chapter11', 0), ('chapter08', 0), ('chapter10', 0), ('chapter09', 0), ('chapter14', 0), ('chapter15', 0), ('chapter16', 0), ('chapter17', 0), ('chapter13', 0), ('chapter18', 0), ('chapter19', 0)]


In [12]:
#printing shapes where they exist
#shapes for each chapter and each json file seem to be exact copy across the json files. frame and character name is same.
print(ch[[0,22,25,27]])
for chapter in ch[[0,22,25,27]]:
  print(len([c['shapes'] for c in all_data[chapter]]), [c['shapes'] for c in all_data[chapter]])

['chapter01' 'chapter23' 'chapter26' 'chapter28']
4 [[{'type': 'rectangle', 'occluded': False, 'z_order': 0, 'points': [662.43, 34.99, 941.33, 333.38], 'id': 137882, 'frame': 6132, 'label_id': 2819, 'group': 0, 'attributes': [{'spec_id': 7845, 'value': 'Eve'}]}], [{'type': 'rectangle', 'occluded': False, 'z_order': 0, 'points': [662.43, 34.99, 941.33, 333.38], 'id': 137876, 'frame': 6132, 'label_id': 2771, 'group': 0, 'attributes': [{'spec_id': 7701, 'value': 'Eve'}]}], [{'type': 'rectangle', 'occluded': False, 'z_order': 0, 'points': [662.43, 34.99, 941.33, 333.38], 'id': 137875, 'frame': 6132, 'label_id': 2755, 'group': 0, 'attributes': [{'spec_id': 7659, 'value': 'Eve'}]}], [{'type': 'rectangle', 'occluded': False, 'z_order': 0, 'points': [662.43, 34.99, 941.33, 333.38], 'id': 137881, 'frame': 6132, 'label_id': 2740, 'group': 0, 'attributes': [{'spec_id': 7614, 'value': 'Eve'}]}]]
3 [[{'type': 'rectangle', 'occluded': False, 'z_order': 0, 'points': [592.22, 0.0, 811.77, 163.84], 'id

Above EDA proves that only 'tracks' is the JSON key which seems to have any meaningful data, rest keys are empty or don't have any meaningful data.

In [13]:
chapter_seq_len = {}
for chapter in all_data:
  chapter_seq_len[chapter] = [len(all_data[chapter][i]['tracks']) for i in range(len(all_data[chapter]))]
#chapter_seq_len
sorted(chapter_seq_len.items(), key=lambda x: x[0])

[('chapter01', [62, 62, 62, 62]),
 ('chapter02', [47, 47, 47, 47]),
 ('chapter03', [76, 76, 76, 76]),
 ('chapter04', [2, 2, 2, 2]),
 ('chapter05', [57, 57, 57, 57]),
 ('chapter06', [14, 14, 14, 14]),
 ('chapter07', [48, 48, 48, 48]),
 ('chapter08', [52, 52, 52, 52]),
 ('chapter09', [56, 56, 56, 56]),
 ('chapter10', [15, 15, 15, 15]),
 ('chapter11', [39, 39, 39, 39]),
 ('chapter12', [25, 25, 25, 0]),
 ('chapter13', [39, 39, 39, 0]),
 ('chapter14', [22, 0, 24]),
 ('chapter15', [0, 0, 45]),
 ('chapter16', [24, 24]),
 ('chapter17', [64, 64]),
 ('chapter18', [34, 34]),
 ('chapter19', [58, 58]),
 ('chapter20', [42, 42, 42]),
 ('chapter21', [84, 84, 84]),
 ('chapter22', [47, 47, 47]),
 ('chapter23', [80, 80, 80]),
 ('chapter24', [26, 26, 26]),
 ('chapter25', [48, 48, 48]),
 ('chapter26', [18, 18, 18]),
 ('chapter27', [29, 29, 29]),
 ('chapter28', [49, 49, 49]),
 ('chapter29', [70, 70, 58]),
 ('chapter30', [56, 56, 56]),
 ('chapter31', [34, 34, 34])]

In [14]:
df = []
len_rows = 0
chapter = ch[0]
for pos,json_array in enumerate(all_data[chapter]):
  print('adding to df file:', diff_chapters[chapter][pos])
  df.append(pd.DataFrame(json_array['tracks']))
  len_rows += len(json_array['tracks'])
  print(len(json_array['tracks']))
df = pd.concat(df)
print(len_rows)

adding to df file: skyfall_emotion_angry_viewer127_chapter01.json
62
adding to df file: skyfall_emotion_fear_viewer127_chapter01.json
62
adding to df file: skyfall_emotion_sad_viewer123_chapter01.json
62
adding to df file: skyfall_emotion_happy_viewer139_chapter01.json
62
248


In [15]:
df

id  frame  label_id  group  \
0   547690   1289      2819      0   
1   547691   1711      2819      0   
2   547692   1958      2819      0   
3   547693   2151      2819      0   
4   547694   2388      2819      0   
..     ...    ...       ...    ...   
57  547279   3695      2740      0   
58  547280   3845      2740      0   
59  547281   5302      2740      0   
60  547282   5302      2740      0   
61  547283   6283      2740      0   

                                               shapes  \
0   [{'type': 'rectangle', 'occluded': False, 'z_o...   
1   [{'type': 'rectangle', 'occluded': False, 'z_o...   
2   [{'type': 'rectangle', 'occluded': False, 'z_o...   
3   [{'type': 'rectangle', 'occluded': False, 'z_o...   
4   [{'type': 'rectangle', 'occluded': False, 'z_o...   
..                                                ...   
57  [{'type': 'rectangle', 'occluded': False, 'z_o...   
58  [{'type': 'rectangle', 'occluded': False, 'z_o...   
59  [{'type': 'rectangle', 'occluded': False, 'z_o...   
60  [{'type': 'rectangle', 'occluded': False, 'z_o...   
61  [{'type': 'rectangle', 'occluded': False, 'z_o...   

                               attributes  
0   [{'spec_id': 7845, 'value': 'James'}]  
1   [{'spec_id': 7845, 'value': 'James'}]  
2   [{'spec_id': 7845, 'value': 'James'}]  
3   [{'spec_id': 7845, 'value': 'James'}]  
4   [{'spec_id': 7845, 'value': 'James'}]  
..                                    ...  
57   [{'spec_id': 7614, 'value': 'Bill'}]  
58   [{'spec_id': 7614, 'value': 'Bill'}]  
59    [{'spec_id': 7614, 'value': 'Eve'}]  
60  [{'spec_id': 7614, 'value': 'James'}]  
61   [{'spec_id': 7614, 'value': 'Bill'}]  

[248 rows x 6 columns]

In [16]:
set(df['label_id'])

{2740, 2755, 2771, 2819}

In [17]:
shape_df = []
for row in df[df['label_id'].isin(set(df['label_id'])) & (df['frame'] == min(df['frame']))].shapes:
  shape_df.append(pd.DataFrame(row))
shape_df = pd.concat(shape_df)
print(len(set(shape_df.frame)), len(set(shape_df.id)))
shape_df
#shape_df[shape_df.frame == min(shape_df.frame)]


7 28


type  occluded  z_order                           points        id  \
0  rectangle     False        0   [430.95, 14.9, 791.54, 421.63]  11942531   
1  rectangle     False        0     [369.42, 8.1, 757.4, 433.59]  11942532   
2  rectangle     False        0   [333.53, 7.25, 721.51, 432.74]  11942533   
3  rectangle     False        0   [283.97, 8.96, 671.95, 434.45]  11942534   
4  rectangle     False        0    [317.22, 8.16, 705.2, 433.65]  11942535   
5  rectangle     False        0  [368.57, 10.67, 756.55, 436.16]  11942536   
6  rectangle     False        0  [368.57, 10.67, 756.55, 436.16]  11942537   
0  rectangle     False        0   [430.95, 14.9, 791.54, 421.63]  11921943   
1  rectangle     False        0     [369.42, 8.1, 757.4, 433.59]  11921944   
2  rectangle     False        0   [333.53, 7.25, 721.51, 432.74]  11921945   
3  rectangle     False        0   [283.97, 8.96, 671.95, 434.45]  11921946   
4  rectangle     False        0    [317.22, 8.16, 705.2, 433.65]  11921947   
5  rectangle     False        0  [368.57, 10.67, 756.55, 436.16]  11921948   
6  rectangle     False        0  [368.57, 10.67, 756.55, 436.16]  11921949   
0  rectangle     False        0   [430.95, 14.9, 791.54, 421.63]  11917430   
1  rectangle     False        0     [369.42, 8.1, 757.4, 433.59]  11917431   
2  rectangle     False        0   [333.53, 7.25, 721.51, 432.74]  11917432   
3  rectangle     False        0   [283.97, 8.96, 671.95, 434.45]  11917433   
4  rectangle     False        0    [317.22, 8.16, 705.2, 433.65]  11917434   
5  rectangle     False        0  [368.57, 10.67, 756.55, 436.16]  11917435   
6  rectangle     False        0  [368.57, 10.67, 756.55, 436.16]  11917436   
0  rectangle     False        0   [430.95, 14.9, 791.54, 421.63]  11938434   
1  rectangle     False        0     [369.42, 8.1, 757.4, 433.59]  11938435   
2  rectangle     False        0   [333.53, 7.25, 721.51, 432.74]  11938436   
3  rectangle     False        0   [283.97, 8.96, 671.95, 434.45]  11938437   
4  rectangle     False        0    [317.22, 8.16, 705.2, 433.65]  11938438   
5  rectangle     False        0  [368.57, 10.67, 756.55, 436.16]  11938439   
6  rectangle     False        0  [368.57, 10.67, 756.55, 436.16]  11938440   

   frame  outside attributes  
0   1289    False         []  
1   1304    False         []  
2   1358    False         []  
3   1375    False         []  
4   1427    False         []  
5   1446    False         []  
6   1456     True         []  
0   1289    False         []  
1   1304    False         []  
2   1358    False         []  
3   1375    False         []  
4   1427    False         []  
5   1446    False         []  
6   1456     True         []  
0   1289    False         []  
1   1304    False         []  
2   1358    False         []  
3   1375    False         []  
4   1427    False         []  
5   1446    False         []  
6   1456     True         []  
0   1289    False         []  
1   1304    False         []  
2   1358    False         []  
3   1375    False         []  
4   1427    False         []  
5   1446    False         []  
6   1456     True         []

In [18]:
f = open(os.path.join(emotion_annotations_f, emotion_annotations[0]))
data = json.load(f)
print(emotion_annotations[0])

json_formatted_str = json.dumps(data, indent=2)
print(json_formatted_str)


skyfall_emotion_disgust_viewer142_chapter20.json
{
  "tags": [],
  "shapes": [],
  "version": 11,
  "tracks": [
    {
      "id": 549003,
      "frame": 471,
      "label_id": 2831,
      "group": 0,
      "shapes": [
        {
          "type": "rectangle",
          "occluded": false,
          "z_order": 0,
          "points": [
            17.44,
            68.2,
            147.02,
            250.26
          ],
          "id": 11963140,
          "frame": 471,
          "outside": false,
          "attributes": [
            {
              "spec_id": 7882,
              "value": "disgust"
            },
            {
              "spec_id": 7883,
              "value": "2"
            }
          ]
        },
        {
          "type": "rectangle",
          "occluded": false,
          "z_order": 0,
          "points": [
            37.79,
            69.27,
            167.37,
            251.33
          ],
          "id": 11963141,
          "frame": 472,
          "outs

In [19]:
#data
# Create an empty array
#print(data["tracks"])
label_frames = []
for track in data["tracks"]:
    shapes = track["shapes"]
    frame = shapes[0]["frame"]
    attributes = shapes[0]["attributes"]
    if attributes != []:
        emotion = attributes[0]["value"]
        label_frames.append((emotion, frame))

def find_last_frame(data):
    last_frame = 0
    for track in data["tracks"]:
        shapes = track["shapes"][-1]
        frame = shapes["frame"]
        last_frame = frame
    return last_frame

def create_tuple_array(data):
    emotions_frames = []
    current_emotion = "not_angry" #get last emotion from previous dataset
    for i in range(find_last_frame(data)):
        if i in label_frames:
            current_emotion = label_frames[0]
        pair = (current_emotion, i)
        emotions_frames.append(pair)
    return emotions_frames
print(len(create_tuple_array(data)))

8146


In [20]:
set(np.array(create_tuple_array(data))[:,0])

{'not_angry'}

In [21]:
# Load in neural data
# NOTE: Takes a while!
neural_data_folder = '/content/drive/MyDrive/Movie Data/e0018RI/2023-04-02_e0018RI_00'

neural_data_dir = os.listdir(neural_data_folder)

neural_data = {}
channels = []
for f in neural_data_dir:
  channel_name = f[f.rfind('_')+1:-5]
  channels.append(channel_name)
  # channel_data = decompress_pickle(os.path.join(neural_data_folder, f))
  # neural_data[channel_name] = channel_data

print(channels)

['Events', 'PULV15', 'PULV14', 'PULV13', 'PULV12', 'PULV11', 'PULV10', 'PULV9', 'PULV8', 'PULV7', 'PULV6', 'PULV5', 'PULV4', 'PULV3', 'PULV2', 'PULV1', 'TPOCC10', 'TPOCC9', 'TPOCC8', 'TPOCC7', 'TPOCC6', 'TPOCC5', 'TPOCC4', 'TPOCC3', 'TPOCC2', 'TPOCC1', 'TOCC12', 'TOCC11', 'TOCC10', 'TOCC9', 'TOCC8', 'TOCC7', 'TOCC6', 'TOCC5', 'TOCC4', 'TOCC3', 'TOCC2', 'TOCC1', 'POCC10', 'POCC9', 'POCC8', 'POCC7', 'POCC6', 'POCC5', 'POCC4', 'POCC3', 'POCC2', 'POCC1', 'OSLAT10', 'OSLAT9', 'OSLAT8', 'OSLAT7', 'OSLAT6', 'OSLAT5', 'OSLAT4', 'OSLAT3', 'OSLAT2', 'OSLAT1', 'OSUP15', 'OSUP14', 'OSUP13', 'OSUP12', 'OSUP11', 'OSUP10', 'OSUP9', 'OSUP8', 'OSUP7', 'OSUP6', 'OSUP5', 'OSUP4', 'OSUP3', 'OSUP2', 'OSUP1', 'OLAT15', 'OLAT14', 'OLAT13', 'OLAT12', 'OLAT11', 'OLAT10', 'OLAT9', 'OLAT8', 'OLAT7', 'OLAT6', 'OLAT5', 'OLAT4', 'OLAT3', 'OLAT2', 'OLAT1']


In [22]:
skyfall_cfg2 = loadmat('/content/drive/MyDrive/Movie Data/e0018RI/skyfall_cfg2.mat')
patient_skyfall = loadmat('/content/drive/MyDrive/Movie Data/e0018RI/2023-04-02_e0018RI_skyfall.mat')

#both look like irrelevant files

/usr/local/lib/python3.10/dist-packages/scipy/io/matlab/_mio.py:227: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


Points:
1. Alex, Torsten were hopeful with the current annotation data. Try to redump the annotations because we have acces to Alekh website. Not more than 1 hour.
2. DEAP dataset -> labels and spectrograms. 2-3 hrs. We'll reuse Josh's VGG code.
3. Silver lining -> Include the analysis of the data in our report. Constructive criticism of the EEG (spikes between chapters) and the labels.

From the professor's persepctive, the data doesn't matter.

# Emotion wise processing of chapters

* Connect all chapters and all emotion files across chapters. Make a big sequence of frames, emotions.
* diff_emotions

In [23]:
for k in diff_emotions:
  diff_emotions[k] = sorted(diff_emotions[k], key = lambda x: x.split('_')[4])

In [24]:
print("Total different EMOTIONS for which labels are available along with their count: ")
print("Total EMOTIONS: ", len(diff_emotions.keys()))
for emotion in sorted(list(diff_emotions.keys())):
  print(emotion, ", number of label files: ", len(diff_emotions[emotion]), end=',')
  print("Files:", sorted(diff_emotions[emotion]))

Total different EMOTIONS for which labels are available along with their count: 
Total EMOTIONS:  5
angry , number of label files:  31,Files: ['skyfall_emotion_angry_viewer127_chapter01.json', 'skyfall_emotion_angry_viewer127_chapter02.json', 'skyfall_emotion_angry_viewer127_chapter03.json', 'skyfall_emotion_angry_viewer127_chapter04.json', 'skyfall_emotion_angry_viewer127_chapter05.json', 'skyfall_emotion_angry_viewer127_chapter06.json', 'skyfall_emotion_angry_viewer127_chapter07.json', 'skyfall_emotion_angry_viewer127_chapter08.json', 'skyfall_emotion_angry_viewer127_chapter09.json', 'skyfall_emotion_angry_viewer127_chapter10.json', 'skyfall_emotion_angry_viewer127_chapter11.json', 'skyfall_emotion_angry_viewer127_chapter12.json', 'skyfall_emotion_angry_viewer127_chapter13.json', 'skyfall_emotion_angry_viewer127_chapter14.json', 'skyfall_emotion_angry_viewer127_chapter15.json', 'skyfall_emotion_angry_viewer127_chapter16.json', 'skyfall_emotion_angry_viewer127_chapter17.json', 'skyfal

In [25]:
#new dataframe which merges all the json files that we have across emotions, then chapters, then json rows['tracks'], then shape rows withing each track row
emotion_df = {'emotion_file': [], 'json_id': [], 'json': [], 'chapter_num': [], 'json_row_id': [], 'track_frame_number': [], \
              'shape_id': [], 'shape_frame_number': [], 'emotion_value': [], 'character_val': []}
all_possible_em_attributes = []
for emotion in diff_emotions:
  for file_ in diff_emotions[emotion]:
    json_array = json.load(open(os.path.join(emotion_annotations_f, file_)))['tracks']
    for track in json_array:
      for shape in track['shapes']:
        #value from loop 1
        emotion_df['emotion_file'].append(emotion) #emotion in the file name of a json file
        #value from loop 2
        emotion_df['json'].append(file_) #name of the json file
        emotion_df['chapter_num'].append(int(file_.split('_')[4][-7:-5])) #name of the json file
        #value from loop 3
        emotion_df['track_frame_number'].append(track['frame']) #frame in each row of json file
        emotion_df['json_row_id'].append(track['id']) #id for each row of a single json file
        emotion_df['json_id'].append(track['label_id']) #id for each json file
        emotion_df['character_val'].append(track['attributes'][0]['value'])  #'attributes': [{'spec_id': 7845, 'value': 'James'}]
        #value from loop 4
        #json array of shapes within the parent json's tracks key
        emotion_df['shape_frame_number'].append(shape['frame'])
        emotion_df['shape_id'].append(shape['id'])
        emotion_df['emotion_value'].append([x['value'] for x in shape['attributes']])  #"attributes": [{"spec_id": 7883, "value": "N_A"}, {"spec_id": 7882, "value": "not_disgust"}
        all_possible_em_attributes.extend([x['value'] for x in shape['attributes']])
emotion_df = pd.DataFrame(emotion_df)

In [26]:
set(all_possible_em_attributes)

{'1',
 '10',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'N_A',
 'angry',
 'disgust',
 'fear',
 'happy',
 'not_angry',
 'not_disgust',
 'not_fear',
 'not_happy',
 'not_sad',
 'sad'}

In [27]:
emotion_df

emotion_file  json_id                                              json  \
0          disgust     2831  skyfall_emotion_disgust_viewer142_chapter20.json   
1          disgust     2831  skyfall_emotion_disgust_viewer142_chapter20.json   
2          disgust     2831  skyfall_emotion_disgust_viewer142_chapter20.json   
3          disgust     2831  skyfall_emotion_disgust_viewer142_chapter20.json   
4          disgust     2831  skyfall_emotion_disgust_viewer142_chapter20.json   
...            ...      ...                                               ...   
41972        happy     2726    skyfall_emotion_happy_viewer139_chapter15.json   
41973        happy     2726    skyfall_emotion_happy_viewer139_chapter15.json   
41974        happy     2726    skyfall_emotion_happy_viewer139_chapter15.json   
41975        happy     2726    skyfall_emotion_happy_viewer139_chapter15.json   
41976        happy     2726    skyfall_emotion_happy_viewer139_chapter15.json   

       chapter_num  json_row_id  track_frame_number  shape_id  \
0               20       549003                 471  11963140   
1               20       549003                 471  11963141   
2               20       549003                 471  11963142   
3               20       549003                 471  11963143   
4               20       549003                 471  11963144   
...            ...          ...                 ...       ...   
41972           15       546368                7876  11928173   
41973           15       546368                7876  11928174   
41974           15       546368                7876  11928175   
41975           15       546368                7876  11928176   
41976           15       546368                7876  11928177   

       shape_frame_number emotion_value character_val  
0                     471  [disgust, 2]         James  
1                     472            []         James  
2                     475            []         James  
3                     476            []         James  
4                     480            []         James  
...                   ...           ...           ...  
41972                7981            []         James  
41973                7983            []         James  
41974                7988            []         James  
41975                7992            []         James  
41976                7998            []         James  

[41977 rows x 10 columns]

In [28]:
#rows where emotion_value is not an empty list
filled_emotions = emotion_df[emotion_df['emotion_value'].apply(lambda x: len(x))>0]
filled_emotions

emotion_file  json_id                                              json  \
0          disgust     2831  skyfall_emotion_disgust_viewer142_chapter20.json   
23         disgust     2831  skyfall_emotion_disgust_viewer142_chapter20.json   
32         disgust     2831  skyfall_emotion_disgust_viewer142_chapter20.json   
120        disgust     2831  skyfall_emotion_disgust_viewer142_chapter20.json   
123        disgust     2831  skyfall_emotion_disgust_viewer142_chapter20.json   
...            ...      ...                                               ...   
41917        happy     2726    skyfall_emotion_happy_viewer139_chapter15.json   
41919        happy     2726    skyfall_emotion_happy_viewer139_chapter15.json   
41921        happy     2726    skyfall_emotion_happy_viewer139_chapter15.json   
41924        happy     2726    skyfall_emotion_happy_viewer139_chapter15.json   
41950        happy     2726    skyfall_emotion_happy_viewer139_chapter15.json   

       chapter_num  json_row_id  track_frame_number  shape_id  \
0               20       549003                 471  11963140   
23              20       549003                 471  11963163   
32              20       549003                 471  11963172   
120             20       549015                2624  11963174   
123             20       549015                2624  11963177   
...            ...          ...                 ...       ...   
41917           15       546364                7774  11928118   
41919           15       546365                7782  11928120   
41921           15       546366                7808  11928122   
41924           15       546367                7859  11928125   
41950           15       546368                7876  11928151   

       shape_frame_number       emotion_value character_val  
0                     471        [disgust, 2]         James  
23                    568  [not_disgust, N_A]         James  
32                    608  [N_A, not_disgust]         James  
120                  2624  [not_disgust, N_A]        Gareth  
123                  2681        [disgust, 2]        Gareth  
...                   ...                 ...           ...  
41917                7774    [N_A, not_happy]         James  
41919                7782    [not_happy, N_A]           Eve  
41921                7808    [not_happy, N_A]         James  
41924                7859    [not_happy, N_A]           Eve  
41950                7876    [N_A, not_happy]         James  

[568 rows x 10 columns]

## Global/across characters emotion label extraction for each chapter and frame:

In [29]:
frame_level = emotion_df.groupby(['chapter_num', 'shape_frame_number']).agg({'emotion_value': lambda x: np.unique(x), 'emotion_file': lambda x: np.unique(x), \
                                                               'character_val': lambda x: np.unique(x)}).reset_index()
frame_level['emotion_value'] = frame_level.emotion_value.apply(lambda x: [i for i in x if len(i)>0])
frame_level

chapter_num  shape_frame_number emotion_value  \
0                1                1289            []   
1                1                1304            []   
2                1                1358            []   
3                1                1375            []   
4                1                1427            []   
...            ...                 ...           ...   
13938           31                3890            []   
13939           31                3922            []   
13940           31                3978            []   
13941           31                4036            []   
13942           31                4150            []   

                    emotion_file    character_val  
0      [angry, fear, happy, sad]          [James]  
1      [angry, fear, happy, sad]          [James]  
2      [angry, fear, happy, sad]          [James]  
3      [angry, fear, happy, sad]          [James]  
4      [angry, fear, happy, sad]          [James]  
...                          ...              ...  
13938     [angry, disgust, fear]         [Gareth]  
13939     [angry, disgust, fear]          [James]  
13940     [angry, disgust, fear]  [Gareth, James]  
13941     [angry, disgust, fear]  [Gareth, James]  
13942     [angry, disgust, fear]  [Gareth, James]  

[13943 rows x 5 columns]

In [67]:
temp_df = frame_level[frame_level.character_val.apply(lambda x: 'Eve' in x)]
temp_df[temp_df.chapter_num.apply(lambda x: x)==8]

chapter_num  shape_frame_number emotion_value  \
3121            8                6339            []   
3122            8                6346            []   
3123            8                6358            []   
3124            8                6377            []   
3125            8                6403            []   
...           ...                 ...           ...   
3326            8                8134            []   
3327            8                8139            []   
3328            8                8142            []   
3329            8                8146            []   
3330            8                8147            []   

                   emotion_file character_val  
3121  [angry, fear, happy, sad]  [Eve, James]  
3122  [angry, fear, happy, sad]         [Eve]  
3123  [angry, fear, happy, sad]         [Eve]  
3124  [angry, fear, happy, sad]         [Eve]  
3125  [angry, fear, happy, sad]         [Eve]  
...                         ...           ...  
3326  [angry, fear, happy, sad]         [Eve]  
3327  [angry, fear, happy, sad]         [Eve]  
3328  [angry, fear, happy, sad]         [Eve]  
3329                     [fear]         [Eve]  
3330  [angry, fear, happy, sad]         [Eve]  

[107 rows x 5 columns]

## Character wise emotion label extraction for each chapter and frame:

In [31]:
#first chapter with char's occurence and with the first emotion label for that char
first_occur = emotion_df.groupby(['character_val'])\
                .agg({'chapter_num':'min'})
first_emo = filled_emotions.groupby(['character_val'])\
                .agg({'chapter_num':'min'})\
                .rename(columns={'chapter_num':'emo_chapter'})
pd.merge(first_occur, first_emo, on='character_val')


chapter_num  emo_chapter
character_val                          
Bill                     1            3
Eve                      1            1
Gareth                   5            5
James                    1            1
M                        1            3
None                     5           15
Raoul                   17           17

In [32]:
#emotion_df partitioned by the character of the movie. It will have all cols of table.
james_df = emotion_df[emotion_df.character_val == 'James']
gareth_df = emotion_df[emotion_df.character_val == 'Gareth']
eve_df = emotion_df[emotion_df.character_val == 'Eve']
bill_df = emotion_df[emotion_df.character_val == 'Bill']
none_df = emotion_df[emotion_df.character_val == 'None']
raoul_df = emotion_df[emotion_df.character_val == 'Raoul']
m_df = emotion_df[emotion_df.character_val == 'M']

In [33]:
#(chapter_num,shape_frame_number) -> emotion_values
james_df_grouped = james_df.groupby(['chapter_num', 'shape_frame_number']).agg({'emotion_value': lambda x: np.unique(x), 'emotion_file': lambda x: np.unique(x)}).reset_index()
gareth_df_grouped = gareth_df.groupby(['chapter_num', 'shape_frame_number']).agg({'emotion_value': lambda x: np.unique(x), 'emotion_file': lambda x: np.unique(x)}).reset_index()
eve_df_grouped = eve_df.groupby(['chapter_num', 'shape_frame_number']).agg({'emotion_value': lambda x: np.unique(x), 'emotion_file': lambda x: np.unique(x)}).reset_index()
bill_df_grouped = bill_df.groupby(['chapter_num', 'shape_frame_number']).agg({'emotion_value': lambda x: np.unique(x), 'emotion_file': lambda x: np.unique(x)}).reset_index()
none_df_grouped = none_df.groupby(['chapter_num', 'shape_frame_number']).agg({'emotion_value': lambda x: np.unique(x), 'emotion_file': lambda x: np.unique(x)}).reset_index()
raoul_df_grouped = raoul_df.groupby(['chapter_num', 'shape_frame_number']).agg({'emotion_value': lambda x: np.unique(x), 'emotion_file': lambda x: np.unique(x)}).reset_index()
m_df_grouped = m_df.groupby(['chapter_num', 'shape_frame_number']).agg({'emotion_value': lambda x: np.unique(x), 'emotion_file': lambda x: np.unique(x)}).reset_index()

char_dfs = {'james': james_df_grouped, 'gareth': gareth_df_grouped, 'bill': bill_df_grouped, 'eve': eve_df_grouped, 'none': none_df_grouped, 'raoul': raoul_df_grouped,\
            'm': m_df_grouped}

#remove the useless [] in the emo values
for _,df in char_dfs.items():
  df['emotion_value'] = df.emotion_value.apply(lambda x: [i for i in x if len(i)>0])

In [34]:
for _,df in char_dfs.items():
  print(_,set(df.emotion_value.apply(lambda x: len(x)))) #same frame can have 0, 1, 2 or 3 labels

james {0, 1, 2, 3}
gareth {0, 1, 2, 3}
bill {0, 1, 2, 3}
eve {0, 1, 2}
none {0, 1}
raoul {0, 1, 2}
m {0, 1, 2, 3}


In [35]:
james_df_grouped

chapter_num  shape_frame_number emotion_value               emotion_file
0               1                1289            []  [angry, fear, happy, sad]
1               1                1304            []  [angry, fear, happy, sad]
2               1                1358            []  [angry, fear, happy, sad]
3               1                1375            []  [angry, fear, happy, sad]
4               1                1427            []  [angry, fear, happy, sad]
...           ...                 ...           ...                        ...
7892           31                3785            []     [angry, disgust, fear]
7893           31                3922            []     [angry, disgust, fear]
7894           31                3978            []     [angry, disgust, fear]
7895           31                4036            []     [angry, disgust, fear]
7896           31                4150            []     [angry, disgust, fear]

[7897 rows x 4 columns]

In [36]:
james_df_grouped[james_df_grouped.emotion_value.apply(lambda x: len(x))==1]

chapter_num  shape_frame_number         emotion_value  \
196             1                4991     [[not_fear, N_A]]   
265             1                5827     [[not_fear, N_A]]   
738             3                 144    [[not_angry, N_A]]   
777             3                 202          [[angry, 5]]   
1102            3                3236    [[not_angry, N_A]]   
...           ...                 ...                   ...   
7674           30                3866        [[disgust, 5]]   
7702           30                4889  [[not_disgust, N_A]]   
7703           30                4947        [[disgust, 2]]   
7717           30                5064     [[not_fear, N_A]]   
7732           30                5170           [[fear, 4]]   

                   emotion_file  
196   [angry, fear, happy, sad]  
265   [angry, fear, happy, sad]  
738   [angry, fear, happy, sad]  
777   [angry, fear, happy, sad]  
1102  [angry, fear, happy, sad]  
...                         ...  
7674                  [disgust]  
7702     [angry, disgust, fear]  
7703                  [disgust]  
7717     [angry, disgust, fear]  
7732     [angry, disgust, fear]  

[233 rows x 4 columns]

In [37]:
james_df_grouped[james_df_grouped.emotion_value.apply(lambda x: len(x))==2]

chapter_num  shape_frame_number                           emotion_value  \
178             1                4332     [[not_angry, N_A], [not_fear, N_A]]   
1341            6                2212     [[not_angry, N_A], [not_fear, N_A]]   
1354            6                2624             [[not_fear, N_A], [sad, 4]]   
1719            8                2169            [[1, sad], [not_angry, N_A]]   
1728            8                2253      [[not_angry, N_A], [not_sad, N_A]]   
2185           10                2470     [[not_angry, N_A], [not_fear, N_A]]   
2254           11                1770     [[not_angry, N_A], [not_fear, N_A]]   
2427           11                5819     [[not_angry, N_A], [not_fear, N_A]]   
2770           14                 287    [[not_angry, N_A], [not_happy, N_A]]   
2776           14                 634    [[N_A, not_happy], [not_angry, N_A]]   
2782           14                 819    [[not_angry, N_A], [not_happy, N_A]]   
2880           14                3100    [[not_angry, N_A], [not_happy, N_A]]   
2906           14                3401    [[N_A, not_happy], [not_angry, N_A]]   
2908           14                3756    [[not_angry, N_A], [not_happy, N_A]]   
2915           14                3879    [[not_angry, N_A], [not_happy, N_A]]   
3043           14                6186    [[not_angry, N_A], [not_happy, N_A]]   
4568           21                 495     [[not_angry, N_A], [not_fear, N_A]]   
5142           21                5310     [[not_angry, N_A], [not_fear, N_A]]   
5547           23                 901     [[N_A, not_fear], [not_angry, N_A]]   
5604           23                2113     [[not_angry, N_A], [not_fear, N_A]]   
5628           23                3300     [[not_angry, N_A], [not_fear, N_A]]   
6354           27                2009     [[not_angry, N_A], [not_fear, N_A]]   
6357           27                2639   [[not_disgust, N_A], [not_fear, N_A]]   
7033           29                 747  [[not_angry, N_A], [not_disgust, N_A]]   
7074           29                 914  [[not_angry, N_A], [not_disgust, N_A]]   

                   emotion_file  
178   [angry, fear, happy, sad]  
1341  [angry, fear, happy, sad]  
1354  [angry, fear, happy, sad]  
1719  [angry, fear, happy, sad]  
1728  [angry, fear, happy, sad]  
2185  [angry, fear, happy, sad]  
2254              [angry, fear]  
2427              [angry, fear]  
2770             [angry, happy]  
2776             [angry, happy]  
2782             [angry, happy]  
2880             [angry, happy]  
2906             [angry, happy]  
2908             [angry, happy]  
2915             [angry, happy]  
3043             [angry, happy]  
4568              [angry, fear]  
5142              [angry, fear]  
5547              [angry, fear]  
5604              [angry, fear]  
5628     [angry, disgust, fear]  
6354              [angry, fear]  
6357     [angry, disgust, fear]  
7033           [angry, disgust]  
7074           [angry, disgust]

Label cleaning is done, for each character we have a dataframe with
chapter num, frame num and the corresponding emotion labels seen in that frame

In [43]:
char_dfs['eve'][char_dfs['eve'].emotion_value.apply(lambda x: len(x)>0)]

chapter_num  shape_frame_number                        emotion_value  \
23             1                4332  [[not_angry, N_A], [not_fear, N_A]]   
28             1                4460                   [[not_angry, N_A]]   
48             1                4968                   [[not_angry, N_A]]   
51             1                4991                    [[not_fear, N_A]]   
488            3                6572                           [[sad, 7]]   
490            3                6763                           [[sad, 9]]   
492            3                6847                           [[sad, 9]]   
500            8                6494                         [[happy, 2]]   
502            8                6800                         [[happy, 3]]   
510            8                6908                   [[not_angry, N_A]]   
585            8                7938                   [[not_happy, N_A]]   
591            8                7972                         [[happy, 4]]   
594            8                8106        [[happy, 4], [not_fear, N_A]]   
743           14                2068                   [[not_happy, N_A]]   
749           14                2590                   [[not_happy, N_A]]   
751           14                2611                   [[not_angry, N_A]]   
766           14                6716                   [[not_happy, N_A]]   
770           14                6759                   [[not_angry, N_A]]   
774           15                7645                   [[not_happy, N_A]]   
781           15                7782                   [[not_happy, N_A]]   
783           15                7859                   [[not_happy, N_A]]   
815           23                 887                 [[not_disgust, N_A]]   
816           23                 896                       [[disgust, 2]]   
817           23                 900                 [[not_disgust, N_A]]   
823           23                1648                    [[not_fear, N_A]]   

                  emotion_file  
23   [angry, fear, happy, sad]  
28   [angry, fear, happy, sad]  
48   [angry, fear, happy, sad]  
51   [angry, fear, happy, sad]  
488  [angry, fear, happy, sad]  
490  [angry, fear, happy, sad]  
492  [angry, fear, happy, sad]  
500  [angry, fear, happy, sad]  
502  [angry, fear, happy, sad]  
510  [angry, fear, happy, sad]  
585  [angry, fear, happy, sad]  
591                    [happy]  
594  [angry, fear, happy, sad]  
743                    [happy]  
749                    [happy]  
751                    [angry]  
766                    [happy]  
770                    [angry]  
774                    [happy]  
781                    [happy]  
783                    [happy]  
815     [angry, disgust, fear]  
816                  [disgust]  
817     [angry, disgust, fear]  
823     [angry, disgust, fear]

In [49]:
char_dfs['eve']

chapter_num  shape_frame_number emotion_value               emotion_file
0              1                3566            []  [angry, fear, happy, sad]
1              1                3590            []  [angry, fear, happy, sad]
2              1                3592            []  [angry, fear, happy, sad]
3              1                3594            []  [angry, fear, happy, sad]
4              1                3617            []  [angry, fear, happy, sad]
..           ...                 ...           ...                        ...
945           31                3224            []     [angry, disgust, fear]
946           31                3235            []     [angry, disgust, fear]
947           31                3251            []     [angry, disgust, fear]
948           31                3255            []     [angry, disgust, fear]
949           31                3282            []     [angry, disgust, fear]

[950 rows x 4 columns]

In [56]:
def process_emotion_dict(emotion_dict):
  # Determine the emotion of a current frame using forward-filling
  emotion = None
  for key, value in emotion_dict:
    if value[:3] == 'not':
      continue
    else:
      if emotion == None:
        emotion = key
      else:
        return 'multi'
  if emotion == None:
    return 'none'

In [55]:
eve_dict = char_dfs['eve'].to_dict()

for key in eve_dict:
  eve_dict[key] = list(eve_dict[key].values())

current_eve_emotions = {'angry': 'not_angry', 'happy': 'not_happy', 'disgust': 'not_disgust', 'fear': 'not_fear', 'sad': 'not_sad'}

eve_labels = {}
# keys: (chapter, frame)
# values: emotion

current_chapter = eve_dict['chapter_num'][0]
for frame in len(eve_dict['chapter_num']):

  # Detect new chapter
  if not eve_dict['chapter_num'][frame] == current_chapter:
    # Reset emotions to negative: I don't think they carry over between chapters, by checking DF it would otherwise look like Eve is sad forever after shooting James
    current_chapter = eve_dict['chapter_num'][frame]
    current_eve_emotions = {'angry': 'not_angry', 'happy': 'not_happy', 'disgust': 'not_disgust', 'fear': 'not_fear', 'sad': 'not_sad'}

  # Detect absence of change in emotion
  if not len(eve_dict['emotion_value'][frame]) > 0:
    # Set label
    eve_labels[(eve_dict['chapter_num'][frame], eve_dict['shape_frame_number'][frame])] = process_emotion_dict(current_eve_emotions)

  # Detect change in emotion
  else:
    # Update emotions
    for emotion in eve_dict['emotion_value']:

    # Set label
    eve_labels[(eve_dict['chapter_num'][frame], eve_dict['shape_frame_number'][frame])] = process_emotion_dict(current_eve_emotions)



{'chapter_num': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,